# Data Creation for BLX DB
#### October 2021

Combine crop data, parcel data, GSA data, and Bulletin 118 data to apply: APNs, basins, GSAs to crop polygons

In [1]:
import fiona
import os
import geopandas as gpd
import pandas as pd

In [2]:
!pwd

/home/watermaster/Projects/BLX/GIS/Dream_DB


In [3]:
fiona.listlayers('./raw_data/i15_crop_mapping_2018_gdb/i15_crop_mapping_2018.gdb')

['i15_Crop_Mapping_2018']

In [4]:
fiona.listlayers('./raw_data/Parcels_CA_2014.gdb/')

['CA_PARCELS_STATEWIDE_INFO', 'CA_PARCELS_STATEWIDE']

In [5]:
fiona.listlayers('./raw_data/ca-county-boundaries/CA_Counties/CA_Counties_TIGER2016.dbf')

['CA_Counties_TIGER2016']

In [6]:
fiona.listlayers('./raw_data/B118_2018_GISdata/Geodatabase/B118_v6-1.gdb')

['i08_B118_v6_1']

In [7]:
fiona.listlayers('./raw_data/submittedgsa/GSA_Master.dbf')

['GSA_Master']

## Load in county data to use a mask to chunk other data by individual county. 

In [8]:
gdf_counties = gpd.read_file('./raw_data/ca-county-boundaries/CA_Counties/CA_Counties_TIGER2016.dbf',
                    driver='FileGDB',
                    layer='CA_Counties_TIGER2016')
gdf_counties = gdf_counties.to_crs(3310)

In [9]:
gdf_kern_county = gdf_counties.loc[gdf_counties['NAME']=='Kern']

In [10]:
# gdf_kern_county.crs
# gdf_kern_county.plot()

## Load in crop data

In [146]:
gdf_crop = gpd.read_file('./raw_data/i15_crop_mapping_2018_gdb/i15_crop_mapping_2018.gdb',
                    driver='FileGDB',
                    layer='i15_Crop_Mapping_2018',
                    mask = gdf_kern_county)
gdf_crop = gdf_crop.to_crs(3310)

In [12]:
# gdf_crop.crs
# gdf_crop.plot()
# gdf_crop.columns

## Load in parcel data

In [13]:
gdf_apn = gpd.read_file('./raw_data/Parcels_CA_2014.gdb/',
                        driver='FileGDB',
                        layer='CA_PARCELS_STATEWIDE',
                        mask = gdf_kern_county)
gdf_apn = gdf_apn.to_crs(3310)

In [14]:
# gdf_apn.crs
# gdf_apn.plot()
# gdf_crop.sindex.valid_query_predicates
# gdf_apn.sindex.valid_query_predicates
# gdf_apn.columns

### __gdf_combo__
(spatial intersectional join of __gdf_crop__ with __gdf_apn__)  

In [19]:
gdf_combo = gpd.sjoin(gdf_crop, gdf_apn, how = 'inner', op = 'intersects')

In [20]:
# gdf_combo.columns
# gdf_combo.loc[gdf_combo['UniqueID'] == '1509614'].T#.plot() # test crop row

In [143]:
gdf_apn.geometry

0         MULTIPOLYGON Z (((88732.232 -291253.638 0.000,...
1         MULTIPOLYGON Z (((88609.535 -291388.489 0.000,...
2         MULTIPOLYGON Z (((88690.503 -291353.672 0.000,...
3         MULTIPOLYGON Z (((88895.635 -291531.530 0.000,...
4         MULTIPOLYGON Z (((88860.618 -291350.298 0.000,...
                                ...                        
401270    MULTIPOLYGON Z (((77446.007 -355552.272 0.000,...
401271    MULTIPOLYGON Z (((67607.874 -350889.851 0.000,...
401272    MULTIPOLYGON Z (((69245.274 -353906.193 0.000,...
401273    MULTIPOLYGON Z (((72561.889 -356393.298 0.000,...
401274    MULTIPOLYGON Z (((77279.250 -355556.274 0.000,...
Name: geometry, Length: 401275, dtype: geometry

### __gdf_over_max__ 
(selection of the maximum spatial overlap of __gdf_crop__ with __gdf_apn__,   
practically translates to each __gdf_crop__ row (`uniqueID` is good identifier) being associated with the __gdf_apn__ row (`PARNO` is good identifier) that has the maximum spatial overlap)

In [21]:
gdf_over = gpd.overlay(gdf_crop, gdf_apn, how = 'intersection')
gdf_over['area_overlap'] = gdf_over.geometry.area
gdf_over_max = gdf_over.loc[gdf_over.groupby('UniqueID')['area_overlap'].agg(pd.Series.idxmax)][['UniqueID','PARNO','area_overlap']]

### Merging
__gdf_combo__ with __gdf_over_max__ 

In [22]:
gdf_combo_max_area = gdf_combo.merge(gdf_over_max, left_on = ['UniqueID','PARNO'], right_on = ['UniqueID','PARNO'])

In [23]:
# gdf_combo_max_area.loc[gdf_combo_max_area['UniqueID'] == '1509614'] # test crop row

In [24]:
gdf_combo_max_area.CROPTYP2.unique()

array(['X', 'T9', 'D14', 'P1', 'V', 'D1', 'F11', 'T6', 'T10', 'T30',
       'T18', 'F16', 'YP', 'P6', 'G6', 'D10', 'C7', 'U', 'G2', 'C', 'D12',
       'T19', 'D5', 'D15', 'F1', 'T31', 'T15', 'T21', 'P3', 'F10', 'T4',
       'D3', 'T27', 'F2', 'T16', 'D13', 'D16', 'C6', 'C5', 'C4', 'T20'],
      dtype=object)

## Add metadata

In [26]:
meta_data_dict = pd.read_excel('./crop_metadata.xlsx', sheet_name='formatted',header=None, names =['key', 'value']).set_index('key').T.to_dict('records')[0]

/home/watermaster/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [126]:
meta_data_dict['G']

'Grain and hay crops'

In [28]:
gdf_combo_max_area['crop2018'] = gdf_combo_max_area['CROPTYP2'].map(meta_data_dict)

In [117]:
# gdf_combo_max_area.head().T

## Load in GSA data

In [15]:
gdf_gsa = gpd.read_file('./raw_data/submittedgsa/',
                        driver='FileGDB',
                        layer='GSA_Master',
                        mask = gdf_kern_county)
gdf_gsa = gdf_gsa.to_crs(3310)

In [16]:
# gdf_apn.crs
# gdf_apn.plot()
# gdf_crop.sindex.valid_query_predicates
# gdf_apn.sindex.valid_query_predicates
gdf_gsa.columns

Index(['GSA_ID', 'DWR_GSA_ID', 'GSA_Name', 'Basin', 'Local_ID', 'Posted_DT',
       'GSA_URL', 'POC_Name', 'POC_Phone', 'POC_Email', '90_Days', 'geometry'],
      dtype='object')

## Load in Bulletin 118 basin data

In [17]:
gdf_118 = gpd.read_file('./raw_data/B118_2018_GISdata/Geodatabase/B118_v6-1.gdb',
                        driver='FileGDB',
                        layer='i08_B118_v6_1',
                        mask = gdf_kern_county)
gdf_118 = gdf_118.to_crs(3310)

In [39]:
# gdf_apn.crs
# gdf_apn.plot()
# gdf_crop.sindex.valid_query_predicates
# gdf_apn.sindex.valid_query_predicates
gdf_118.columns

Index(['Basin_Number', 'Basin_Subbasin_Number', 'Basin_Name',
       'Basin_Subbasin_Name', 'Region_Office', 'Date_Record_Last_Edited',
       'Record_Edited_By', 'Comments', 'Date_Data_Applies_To', 'GlobalID',
       'SHAPE_Length', 'SHAPE_Area', 'geometry'],
      dtype='object')

In [199]:
gdf_combo_max_area = gdf_combo_max_area.drop('index_right', axis=1)

## Add B118 information via spatial overlay

In [203]:
gdf_combo_118 = gpd.sjoin(gdf_combo_max_area, gdf_118, how = 'inner', op = 'intersects')

In [5]:
# gdf_combo_118.columns

In [205]:
gdf_over = gpd.overlay(gdf_combo_max_area, gdf_118, how = 'intersection')
gdf_over['area_overlap'] = gdf_over.geometry.area
gdf_over_max = gdf_over.loc[gdf_over.groupby('UniqueID')['area_overlap'].agg(pd.Series.idxmax)][['UniqueID','Basin_Subbasin_Number','area_overlap']]

In [206]:
gdf_combo_118_max_area = gdf_combo_118.merge(gdf_over_max, left_on = ['UniqueID','Basin_Subbasin_Number'], right_on = ['UniqueID','Basin_Subbasin_Number'])

In [4]:
# gdf_combo_118_max_area.columns

In [210]:
gdf_combo_118_max_area = gdf_combo_118_max_area[['UniqueID','geometry', 'PARNO', 'County','Basin_Subbasin_Number', 'crop2018','REGION','ACRES']]

## Add GSA information via spatial overlay

In [212]:
gdf_combo_GSA = gpd.sjoin(gdf_combo_118_max_area, gdf_gsa, how = 'inner', op = 'intersects')

In [3]:
# gdf_combo_GSA.columns

In [215]:
gdf_over = gpd.overlay(gdf_combo_118_max_area, gdf_gsa, how = 'intersection')
gdf_over['area_overlap'] = gdf_over.geometry.area
gdf_over_max = gdf_over.loc[gdf_over.groupby('UniqueID')['area_overlap'].agg(pd.Series.idxmax)][['UniqueID','GSA_ID','area_overlap']]

In [216]:
gdf_combo_118_GSA_max_area = gdf_combo_GSA.merge(gdf_over_max, left_on = ['UniqueID','GSA_ID'], right_on = ['UniqueID','GSA_ID'])

In [2]:
# gdf_combo_118_GSA_max_area.columns

In [222]:
test_export = gdf_combo_118_GSA_max_area.explode()[['UniqueID','geometry', 'PARNO', 'County','GSA_ID', 'DWR_GSA_ID', 'GSA_Name', 'Basin_Subbasin_Number_left', 'crop2018','REGION','ACRES']]

In [223]:
test_export.to_file('test_export%.gpkg' % '_kern', driver='GPKG')  